<a href="https://colab.research.google.com/github/ROLEX-codes/DATA-SCIENCE/blob/main/Road_Accident.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import joblib

In [ ]:
# Load the dataset
data = pd.read_csv('RTA Dataset.csv')

In [ ]:
data.head()


,Time,Day_of_week,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Owner_of_vehicle,Service_year_of_vehicle,...,Vehicle_movement,Casualty_class,Sex_of_casualty,Age_band_of_casualty,Casualty_severity,Work_of_casuality,Fitness_of_casuality,Pedestrian_movement,Cause_of_accident,Accident_severity
0,17:02:00,Monday,18-30,Male,Above high school,Employee,1-2yr,Automobile,Owner,Above 10yr,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Moving Backward,Slight Injury
1,17:02:00,Monday,31-50,Male,Junior high school,Employee,Above 10yr,Public (> 45 seats),Owner,5-10yrs,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury
2,17:02:00,Monday,18-30,Male,Junior high school,Employee,1-2yr,Lorry (41?100Q),Owner,NaN,...,Going straight,Driver or rider,Male,31-50,3,Driver,NaN,Not a Pedestrian,Changing lane to the left,Serious Injury
3,1:06:00,Sunday,18-30,Male,Junior high school,Employee,5-10yr,Public (> 45 seats),Governmental,NaN,...,Going straight,Pedestrian,Female,18-30,3,Driver,Normal,Not a Pedestrian,Changing lane to the right,Slight Injury
4,1:06:00,Sunday,18-30,Male,Junior high school,Employee,2-5yr,NaN,Owner,5-10yrs,...,Going straight,na,na,na,na,NaN,NaN,Not a Pedestrian,Overtaking,Slight Injury


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12316 entries, 0 to 12315
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Age_band_of_driver           12316 non-null  object
 1   Driving_experience           11487 non-null  object
 2   Type_of_vehicle              11366 non-null  object
 3   Area_accident_occured        12077 non-null  object
 4   Lanes_or_Medians             11931 non-null  object
 5   Road_allignment              12174 non-null  object
 6   Types_of_Junction            11429 non-null  object
 7   Road_surface_conditions      12316 non-null  object
 8   Light_conditions             12316 non-null  object
 9   Weather_conditions           12316 non-null  object
 10  Type_of_collision            12161 non-null  object
 11  Number_of_vehicles_involved  12316 non-null  int64 
 12  Number_of_casualties         12316 non-null  int64 
 13  Vehicle_movement             12

In [ ]:
px.pie(data,data['Cause_of_accident'],data['Number_of_casualties'],color='Cause_of_accident',template='ggplot2',hole=0.35)


In [ ]:
# dropping columns that can cause imbalance while imputation
lists=['Vehicle_driver_relation', 'Work_of_casuality', 'Fitness_of_casuality','Day_of_week','Casualty_severity','Time','Sex_of_driver','Educational_level','Defect_of_vehicle','Owner_of_vehicle','Service_year_of_vehicle', 'Road_surface_type','Sex_of_casualty']
data.drop(columns = lists, inplace=True)


In [ ]:
data.groupby('Accident_severity').size()


Accident_severity
Fatal injury        158
Serious Injury     1743
Slight Injury     10415
dtype: int64

In [ ]:
data.dropna(inplace=True)


In [ ]:
for i in data.columns:
    if data[i].dtypes== object:
        print(i)
        print(data[i].unique())
        print(data[i].nunique())
        print()


In [ ]:
# Define dependent and independent variables
X=data.drop('Accident_severity',axis=1)
Y=data['Accident_severity']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



In [ ]:
from sklearn.preprocessing import LabelEncoder

l = LabelEncoder()
for col in data.columns:
    if data[col].dtype == object:
        data[col] = l.fit_transform(data[col])


In [68]:
# Create and train the linear regression model
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler(feature_range=(0,1))

# Identify non-numeric columns and exclude them
non_numeric_columns = X_train.select_dtypes(exclude=['number']).columns

# Verify that there are numeric columns to scale
numeric_columns = X_train.columns.difference(non_numeric_columns)
if len(numeric_columns) > 0:
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()

    # Scale only the numeric columns in X_train and X_test
    X_train_scaled[numeric_columns] = mms.fit_transform(X_train[numeric_columns].values)  # Convert to NumPy array
    X_test_scaled[numeric_columns] = mms.transform(X_test[numeric_columns].values)  # Convert to NumPy array
else:
    print("No numeric columns found for scaling.")











No numeric columns found for scaling.


In [69]:
# Create the RandomForestClassifier
rf = RandomForestClassifier()

# Fit the model using X_train_scaled (features) and Y_train (target variable)
rf.fit(X_train_scaled, Y_train)

# Use the model to make predictions
ypred = rf.predict(X_test_scaled)

ValueError: ignored